In [10]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [44]:
n=10000
x=np.arange(n)
y = np.random.normal(size=(1,10000))+1

In [45]:
y.shape

(1, 10000)

In [46]:
kernel = stats.gaussian_kde(y)

In [ ]:
sns.histplot(y)

In [ ]:
ymin=min(y)
ymax=max(y)
ymin,ymax

In [ ]:
points=np.linspace(ymin,ymax,2000)

In [6]:
from scipy import stats

In [ ]:
x2=np.random.normal(size=(2,512))

In [ ]:
y2=kernel(x2)

In [ ]:
plt.plot(points,y2)